# 05d: Calculate direct impacts

In this script, we calculate direct impacts, meaning impacts from the final life cycle stage. For liquids and hydrogen and gases, this is already done in the scripts that calculate the combustion impacts.

In [7]:
%run common_definitions.py

In [22]:
import pandas as pd
import brightway2 as bw
import xarray as xr
import numpy as np
import csv

import time

In [9]:
bw.projects.set_current(BW_PROJECTNAME)
output_fp = "../output/" + BW_PROJECTNAME

### Get mapping for the technology selection

In [10]:
df = pd.read_csv("../mappings/technology_selection_energy_provision_v4.csv")
long2short = dict(zip(list(df["ecoinvent name"]), list(df["short name"])))

In [11]:
sector_selection = [
    "electricity production",
    "district or industrial heat",
    "residential heating"
]

In [12]:
df = df[df["sector"].isin(sector_selection)].fillna("")
df

,short name,reference product,ecoinvent name,sector,direct process
0,BIGCC,"electricity, high voltage","electricity production, at biomass-fired IGCC ...",electricity production,technosphere_if_RER
1,BIGCC w CCS,"electricity, high voltage","electricity production, at biomass-fired IGCC ...",electricity production,technosphere_if_RER
2,hard coal IGCC,"electricity, high voltage","electricity production, at hard coal-fired IGC...",electricity production,technosphere_if_RER
3,hard coal w CCS (post),"electricity, high voltage","electricity production, at hard coal-fired pow...",electricity production,technosphere_if_RER
4,hard coal IGCC w CCS (pre),"electricity, high voltage","electricity production, at hard coal-fired IGC...",electricity production,technosphere_if_RER
...,...,...,...,...,...
145,light fuel oil heating plant,"heat, district or industrial, other than natur...","heat production, light fuel oil, at industrial...",district or industrial heat,technosphere
146,natural gas heating plant,"heat, district or industrial, natural gas","heat production, natural gas, at boiler conden...",district or industrial heat,technosphere
147,softwood heating plant,"heat, district or industrial, other than natur...","heat production, softwood chips from forest, a...",district or industrial heat,technosphere
148,industry wood chips heating plant,"heat, district or industrial, other than natur...","heat production, wood chips from industry, at ...",district or industrial heat,


In [13]:
def get_dbname(scen, year):
    matches = [db for db in bw.databases if "{}_{}".format(scen, year) in db]

    return matches[0]

### Search for activities

In [14]:
activities = {}
subset_names = list(df["ecoinvent name"].unique())
for scen in SCENARIOS:
    activities[scen] = {}
    for year in YEARS:
        dbname = get_dbname(scen, year)
        db = bw.Database(dbname)
        db_subset = [act for act in db if act["name"] in subset_names]
        activities[scen][year] = {}
        for sector in df["sector"].unique():
            sel = df[df["sector"] == sector]
            all_matches = []
            for name, product in zip(list(sel["ecoinvent name"]), list(sel["reference product"])):
                matches = [act for act in db_subset if act["name"] == name and act["reference product"] == product]
                print("\t{} activities found for {}, {} in database {}".format(len(matches), name, product, dbname))
                all_matches += matches
            print("{} activities found in {} in database {}".format(len(all_matches), sector, dbname))
            activities[scen][year][sector] = all_matches

	14 activities found for electricity production, at biomass-fired IGCC power plant, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at biomass-fired IGCC power plant, pre, pipeline 200km, storage 1000m, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coal-fired IGCC power plant, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coal-fired power plant, post, pipeline 200km, storage 1000m, electricity, high voltage in database ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	14 activities found for electricity production, at hard coa

### Get needed LCIA methods

In [15]:
df2 = pd.read_csv("../data/mfs_all_factors.csv").dropna(subset="LCIA method")

In [16]:
needed_methods = list(df2["LCIA method"].unique())
methods = [m for m in list(bw.methods) if ", ".join(list(m)) in needed_methods]

In [17]:
cc_methods = [
    ("IPCC 2021", "climate change: biogenic", "GWP 100a"),
    ("IPCC 2021", "climate change: land use", "GWP 100a"),
    ("IPCC 2021", "climate change: fossil", "GWP 100a")
]
methods = methods + cc_methods

In [18]:
print("{} methods needed, {} methods found".format(len(needed_methods)+len(cc_methods), len(methods)))

73 methods needed, 73 methods found


In [29]:
methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (high NOx)'),
 ('EDIP 2003', 'acidification', 'acidification'),
 ('EDIP 2003', 'eutrophication', 'combined potential'),
 ('EDIP 2003', 'eutrophication', 'terrestrial eutrophication'),
 ('EDIP 2003', 'photochemical ozone formation', 'impacts on human health'),
 ('EDIP 2003', 'photochemical ozone formation', 'impacts on vegetation'),
 ('EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.1',
  'ecotoxi

### Impact calculation

In [34]:
def get_direct_processes(act, direct_process_type):
    if direct_process_type == "":
        return {act: 1}
    if direct_process_type == "technosphere":
        direct_processes = {}
        for exc in act.technosphere():
            if exc["name"] == act["name"]:
                direct_processes[bw.get_activity(exc["input"])] = exc["amount"]
        return direct_processes
    if direct_process_type == "technosphere_if_RER":
        if act["location"] == "RER":
            direct_processes = {}
            for exc in act.technosphere():
                if exc["name"] == act["name"]:
                    direct_processes[bw.get_activity(exc["input"])] = exc["amount"]
            return direct_processes
        else:
            return {act: 1}
        
def get_biosphere_exchanges(direct_processes):
    exchanges = {}
    for act, factor in direct_processes.items():
        for exc in act.biosphere():
            key = exc["input"][1]
            value = exc["amount"]
            if key in exchanges.keys():
                exchanges[key] += value
            else:
                exchanges[key] = value

    return exchanges

def get_cfs(keys, methods):
    # initialize values
    cfs = np.zeros((len(keys), len(methods)))

    # loop through methods
    for j, m in enumerate(methods):
        # print("Collecting CFs for method {} of {}, {}".format(j+1, len(methods), str(m)))
        for flow in bw.Method(m).load():
            key = flow[0][1]
            if key in keys:
                i = keys.index(key)
                cfs[i,j] = flow[1]

    # construct array
    da = xr.DataArray(
        cfs,
        coords={
            "code": keys,
            "LCIA method": [", ".join(list(m)) for m in methods]
        }
    )

    return da    

In [45]:
dflist = []

for scen in SCENARIOS:
    for year in YEARS:
        print("Calculating direct impacts for scenario {}, year {}".format(scen, year))

        t0 = time.time()

        for sector in df["sector"].unique():
            print("\tSector {}".format(sector))
            mapping = df[df["sector"] == sector]
            direct_processes_dict = dict(zip(mapping["ecoinvent name"], mapping["direct process"]))
            actlist = activities[scen][year][sector]

            print("\t\tGetting biosphere exchanges")
            efs = []
            idxlist = []
            for act in actlist:
                dp = get_direct_processes(act, direct_processes_dict[act["name"]])
                efs.append(pd.Series(get_biosphere_exchanges(dp), dtype="float64"))
                
        
            efs = pd.DataFrame(efs).T.fillna(0)
            efs = xr.DataArray(
                efs.to_numpy(),
                coords={
                    "code": list(efs.index),
                    "activity": list(efs.columns)
                }
            )

            cfs = get_cfs(list(efs.coords["code"].values), methods)

            impacts = (efs * cfs).sum(dim="code").to_pandas().reset_index().drop("activity", axis=1)
            impacts["short name"] = [long2short[act["name"]] for act in actlist]
            impacts["ecoinvent name"] = [act["name"] for act in actlist]
            impacts["sector"] = sector
            impacts["scenario"] = scen
            impacts["year"] = year
            impacts["location"] = [act["location"] for act in actlist]
            impacts["unit"] = [act["unit"] for act in actlist]
            impacts["amount"] = 1
            impacts["product"] = [act["reference product"] for act in actlist]

            dflist.append(impacts)

        t1 = time.time()

        print("\tDone! Calculation took {} seconds".format(t1-t0))

columns_reordered = ["short name", "ecoinvent name", "sector",
        "location", "unit", "amount", "product",
        "scenario", "year"]
columns_reordered += [", ".join(list(m)) for m in methods]

pd.concat(dflist)[columns_reordered].to_csv(output_fp + "/direct_impacts_energy_provision.csv", index=False)

Calculating direct impacts for scenario SSP2-NPi, year 2020
	Sector electricity production
		Getting biosphere exchanges
	Sector district or industrial heat
		Getting biosphere exchanges
	Sector residential heating
		Getting biosphere exchanges
	Done! Calculation took 5.115344762802124 seconds
Calculating direct impacts for scenario SSP2-NPi, year 2030
	Sector electricity production
		Getting biosphere exchanges
	Sector district or industrial heat
		Getting biosphere exchanges
	Sector residential heating
		Getting biosphere exchanges
	Done! Calculation took 3.8617262840270996 seconds
Calculating direct impacts for scenario SSP2-NPi, year 2040
	Sector electricity production
		Getting biosphere exchanges
	Sector district or industrial heat
		Getting biosphere exchanges
	Sector residential heating
		Getting biosphere exchanges
	Done! Calculation took 3.7974467277526855 seconds
Calculating direct impacts for scenario SSP2-NPi, year 2050
	Sector electricity production
		Getting biosphere ex

KeyError: "['reference product'] not in index"